In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
### changes in physics - seasonal avg
sst_delta = 0.5 #change in SST, deg/yr?
mld_delta = 10 #change in MLD, m/yr
ot_delta = 5 # change in OT, Sv/yr (SV = 1e6 m3/s)

#### gradients in BGC
grad_dDICdz = 0.28 # (umol/L/m) average between 0-100m depth and 100-200m
grad_DIC_surfdeep = 120 #umol/L between surface and 1000m depth
grad_dFedz = 0.00007 #(nmol/L/m) (nmol/L/m)
grad_dFev = 0.4 # deep minus surface ocean (nmol/L) between surface and 1000m depth (based on Tagliabue 2012 database)

### mean ecosystem parameters
mu_L = 1 # 0-1 depending on month  (Light limitation; no units)
dmudFe = 0.59 # (1/d/(nmolFe/L)) see worksheet Fer parameters "C-estimations-v2"
diatconc = 1 # mean diatom concentration (umol/L)

### carbonate system variables
mn_kg = 0.065 # (mol/m2/yr/ppm) xx-TJ I don't know what this is
mn_SST = 2 #(oC)
mn_Revfact = 14 # Definition: R=(dCO2/CO2)/(dDIC/DIC)
mn_DIC = 2120 # (umol/L)
mn_TA = 2290 # (ueq/L)
mn_pCO2 = 380 # (ppm)

# conversions/ geography
PgC_mol = 1.2e14 # pgC/mol
SO_area = 5.94e13 # assuming 1/6 of the global ocean south of 50S, calculate this from mesh mask


In [1]:
def effect_SST(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area):
    
    dfco2 = mn_kg*0.04*mn_pCO2*sst_delta #dfCO2 = kg(4%-pCO2/degree x dSST)
    dfco2_pgCyr = dfco2 * PgC_mol * SO_area #dfCO2 in PgC/yr

    return dfco2_pgCyr 

def effect_MLD(mld_delta, grad_dDICdz, mn_Revfact, mn_pCO2, mn_DIC, mn_kg, PgC_mol, SO_area ):
    dDIC = mld_delta*grad_dDICdz #dDIC = dMLD x dDIC/dz, umol/L
    dpCO2 = mn_Revfact * mn_pCO2 / mn_DIC * dDIC #dpCO2 = R x pCO2/DIC x dDIC, ppm
    dfCO2_DIC = mn_kg * dpCO2 #dfCO2_DIC = kg x dpCO2
    dfco2_pgCyr_DIC = dfCO2_DIC * PgC_mol * SO_area 
    dfcCO2_TA = (-1/3)*dfCO2_DIC
    dfco2_pgCyr_TA = dfcCO2_TA * PgC_mol * SO_area 

    return dfco2_pgCyr_DIC, dfco2_pgCyr_TA

def effect_MLD_TA()

